### Allele-specific expression

RNA sequencing can distinguish mRNAs expressed from from different copies of genes on homologous chromosomes when single-nucleotide polymorphisms (perhaps silent) distinguish the two alleles. 

Linkage between these distinctive SNPs and _cis_-regulatory sequences can provide information on regulatory variation within a shared cellular context.

In the example below, the maternally derived copy of the gene has a `TGC` serine codon at the 2nd position and a functioning enhancer. The paternally derived allele has a `TGT` serine codon and a mutated enhancer.

```
Maternal: ----enhancer----ATGTGCACT...

Paternal: ----exhxnxer----ATGTGTACT...
```

Because the paternal enhancer is mutated, we would expect most mRNA to derive from the maternal allele and thus most RNA-Seq reads should have a `C` rather than a `T`.

### Null hypothesis testing

The _null hypothesis_ in allele-specific expression analysis is that the alleles are expressed equally, and so each read is equally likely to be derived from each allele.

Our actual data comes from _counting_ how many RNA-seq reads have the maternally-derived `C` variant and how many have the paternally-derived `T` variant.

We want to build a complex probabilistic model of these read counts from a simpler model:
* each read is equally likely to be a `C` or a `T`
* each read is independent

We will use this to ask — if we see an imbalance between maternally- and paternally-derived alleles — whether this imbalance is very unlikely to occur in our null model that reads are equally likely to come from each allele.

We will do this in two ways: first we will actually perform random sampling, and second we will use a mathematical formula that fits this kind of model.

#### Random sampling

We begin by importing pandas

Now we create a pandas `Series` object containing the two possible alleles.

The _values_ will be the two possible genotypes and the _indexes_ will help us remember that one variant is maternally-derived and the other paternally-derived.

We can use the `sample` method to take one (by default) or more (using the `n=` parameter) random allele choices.

When we're taking more than one random sample, we also need to use the `replace=True` parameter because the default is sampling without replacement, but sampling with replacement — that is, being able to randomly choose a given allele more than once — better models gene expression.

We can then use the `value_counts` method to count up the number of maternal and paternal alleles seen across the randomly sampled reads.

Next we want to run this sampling procedure 10,000 times and count up how often we see zero `T` alleles out of eight reads, how often we see one `T` allele, and so forth.

To do this, we will:
* write a `for` loop to run 10,000 times
* sample alleles from 8 reads
* count `T` alleles in our sample (use the `get` method with default value 0)
* add the count of `T` alleles to a list of sample counts
* after generating all of our sample counts, convert them to a `Series` and find the value counts.

We can divide these by the total number of random samples we generated in order to get a normalized probability.

We can also sort them for easier reading using `sort_index`

We can compute the probability of seeing _exactly_ 2 paternally-derived alleles

We can also compute the probability of seeing _no more than_ 2 paternally-derived alelles

Lastly, we'll use matplotlib to plot this distribution.

Later, we'll want to look at random samples taken with different numbers of reads. We'll also want to control the number of samples we take. To simplify this, we'll package up all of our random sampling code into a function that has parameters to control the number of reads in each sample and the total number of samples.

To test our function, we will sample probabilities for 16 reads.

#### The binomial distribution

Our probabilistic model of counting up the results of several independent trials is very common in probability. From a mathematical perspective, it's the same as counting the number of heads that come up out of _N_ total coin tosses.

This number follows a _binomial distribution_. We can compute
$$P(\;k\;\text{heads out of}\;N\;\text{coin tosses})$$
using
```
binom.pmf(k, N, 0.5)
```
The `0.5` parameter here is the probability of a head for each coin toss — the `binom.pmf` function can compute probabilities for a _biased_ coin where the chances of a head are not exactly $\frac{1}{2}$.

The `binom.pmf` function comes from the `binom` module in `scipy.stats`. We'll import this and then compute a probability under the binomial model for exactly 2 paternally-derived reads out of 8 coin tosses.

This binomial probability is very close to the value we got by random sampling!

We can also construct the whole table of binomial probabilities by using a `range()` for the $k$ parameter.

We can plot the binomial probabilities and the random sampling probabilities on the same graph and compare them.

We can also control the plot styles: I'll use the `'b-'` parameter for blue lines and the `'ro'` parameter for red dots. 

#### More reads give more certainty

In this binomial model — and in other more realistic models of deep sequencing data — greater sequencing depth gives more statistical power to reject the null hypothesis.

To see this effect in action for the binomial model of allele-specific expression, consider two possibilities:
* No more than one quarter of 8 reads are paternally-derived
* No more than one quarter of 16 reads are paternally-derived

In both cases, the _magnitude_ of the paternal vs maternal skew is 1:3.